In [3]:
import tensorflow as tf
import cv2
import pandas as pd
import numpy as np
import seaborn as sns
from tensorflow.keras.layers import Layer

import os

In [5]:

class QuadDense(Layer):
    def __init__(self, units=64, activation = 'relu'):
        super(QuadDense, self).__init__()
        self.units = units
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        w1_init = tf.random_normal_initializer()
        self.w1 = tf.Variable(name = 'kernel_1', initial_value = w1_init(shape = (input_shape[-1], self.units), dtype = 'float32'))
        
        w2_init = tf.random_normal_initializer()
        self.w2 = tf.Variable(name = 'kernel_2', initial_value = w2_init(shape = (input_shape[-1], self.units), dtype = 'float32'))

        bias_init = tf.zeros_initializer()
        self.b = tf.Variable(name = 'bias', initial_value = bias_init(shape = (self.units,), dtype = 'float32'))


    def call(self, inputs):
        return self.activation((tf.matmul(tf.math.square(inputs), self.w1)+tf.matmul(inputs, self.w2)+self.b))

                             


In [8]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  QuadDense(64, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 10s 4ms/step - loss: 0.3361 - accuracy: 0.9007
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1824 - accuracy: 0.9463
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1464 - accuracy: 0.9552
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1285 - accuracy: 0.9609
Epoch 5/5
313/313 [==============================] - 1s 4ms/step - loss: 0.0929 - accuracy: 0.9740


[0.09286390990018845, 0.9739999771118164]

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 quad_dense (QuadDense)      (None, 64)                100416    
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 10)                650       
                                                                 
Total params: 101,066
Trainable params: 101,066
Non-trainable params: 0
_________________________________________________________________
